In [126]:
# importing the lib
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import pandas as pd
import numpy
import time
import sys
from selenium.webdriver.chrome.service import service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
import re
import numpy as np

In [127]:
def kayak(doj,souce_airport,destination_airport):
    url="https://www.ca.kayak.com/flights/" +str(source_airport) +"-" +str(destination_airport)+ "/"+str(doj)+ "?sort=bestflight_a"
    driver = webdriver.Chrome(ChromeDriverManager().install())
    driver.maximize_window()
    driver.get(url)
    time.sleep(15)
    def click_more():
    
        for val in range(0,50):
            try:
                driver.find_element_by_xpath("//a[@class='moreButton']").click()
                time.sleep(2)
                val = val +1

            except:
                pass
    click_more()
    
    # depart-time base-time
    departure_time=[]
    departure_time= driver.find_elements_by_xpath("//ol[@class='flights']//span[@class='depart-time base-time']")

    arrival_time=[]
    arrival_time= driver.find_elements_by_xpath("//ol[@class='flights']//span[@class='arrival-time base-time']")

    duration =[]
    duration  = driver.find_elements_by_xpath("//ol[@class='flights']//div[@class='section duration allow-multi-modal-icons']//div[@class='top']")


    no_stop = []
    no_stop  = driver.find_elements_by_xpath("//ol[@class='flights']//div[@class='section stops']//div[@class='top']")

    price =[]
    price = driver.find_elements_by_xpath("//div[@class = 'booking']")

    airlines = []
    airlines = driver.find_elements_by_xpath("//span[@class = 'codeshares-airline-names']")

    # departure_time = driver.find_elements_by_xpath("//div[@class='itinerary-depart-time depart-time']")

    
    
    
    price_list=[]
    stops=[]
    for i in range(len(departure_time)):
        departure_time[i]= departure_time[i].get_attribute('innerHTML')
        arrival_time[i] = arrival_time[i].get_attribute('innerHTML')
        duration[i] = duration[i].get_attribute('innerHTML')
        no_stop[i] = no_stop[i].get_attribute('innerHTML')
        l = re.findall("(direct|1|2|3|4|5)",no_stop[i])[0]
        stops.append(l)
        price[i] = price[i].get_attribute('innerHTML')

#         airlines[i] = airlines[i].get_attribute('innerHTML')
        b = re.findall("nbsp\;((\d\d\d)|(\d.\d\d\d)|(\d\d.\d\d\d))",price[i])[0][0]
        price_list.append(b)
    
    for k in range(len(airlines)):
        airlines[k] = airlines[k].get_attribute('innerHTML')
        
    # Creating data frames
    departure = len(departure_time)*[str(source_airport)]
    arrival = len(departure_time)*[str(destination_airport)]
    # a = pd.DataFrame(departure_time,arrival_time,price_list)
    doj1 = len(departure_time)*[doj]
    df  = pd.DataFrame(doj1,columns=["Date of journey"])
    df["Source"] = departure    
    df["Destination"] = arrival
    df["Departure_time"] = departure_time
    df["Arrival_time"] = arrival_time
    df["Duration"] = duration
    df["Stops"] = stops
    df["CAD $"]= price_list
    df["Airlines"] = airlines
#     df["Stops"]=df["Stops"].astype(str).str.replace("\n","")
    df["CAD $"]=df["CAD $"].astype(str).str.replace(",","")
    df["CAD $"] = pd.to_numeric(df['CAD $'])
#     df["Stops"]=df["Stops"].astype(str).str.replace("stops","")
#     df["Stops"]=df["Stops"].astype(str).str.replace("stop","")
    df["Duration"]=df["Duration"].astype(str).str.replace("\n","")
    df = df.drop(df[(df.Stops =='3') | (df.Airlines == 'Multiple Airlines')].index)
    df= df.drop_duplicates()
    df = df.sort_values(by='CAD $')
    df = df.reset_index()
    df.index = df.index +1
    df.pop("index")
    driver.close()
    return df
    
    

In [128]:
# source = ["YYZ","YYC","YVR","YQB","YHZ"]
# destination = ["DEL","BOM","BLR","AMD","IXC"]
source = ["YVR"]
destination = ["BOM","AMD"]

doj='2022-05-30'
df1 = pd.DataFrame(columns=['Date of journey', 'Source', 'Destination', 'Departure_time',
                            'Arrival_time', 'Duration', 'Stops', 'CAD $', 'Airlines'])
for val in source:
    for values in destination:
        source_airport = val
        print(source_airport)
        
        
        destination_airport = values
        print(destination_airport)
        doj ='2022-05-30'
        df = kayak(doj,source_airport,destination_airport)
#         print("Departure length",len(departure_time))
#         print(df1.describe())
        df.to_csv('kayak_data.csv', mode='a',index=False,header = False)
#         df1=pd.concat([df1,df])
#         print(df1.shape())
        



====== WebDriver manager ======
Current google-chrome version is 99.0.4844
Get LATEST chromedriver version for 99.0.4844 google-chrome
Driver [C:\Users\dhair\.wdm\drivers\chromedriver\win32\99.0.4844.51\chromedriver.exe] found in cache


YVR
BOM


<ipython-input-127-bb81a9d55b4f>:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())
<ipython-input-127-bb81a9d55b4f>:11: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  driver.find_element_by_xpath("//a[@class='moreButton']").click()
<ipython-input-127-bb81a9d55b4f>:21: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  departure_time= driver.find_elements_by_xpath("//ol[@class='flights']//span[@class='depart-time base-time']")
<ipython-input-127-bb81a9d55b4f>:24: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  arrival_time= driver.find_elements_by_xpath("//ol[@class='flights']//span[@class='arrival-time base-time']")
<ipython-input-127-bb81a9d55b4f>:27: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_e

YVR
AMD


In [129]:
df

,Date of journey,Source,Destination,Departure_time,Arrival_time,Duration,Stops,CAD $,Airlines
1,2022-05-30,YVR,AMD,11:55 a.m.,4:25 a.m.,28h 00m,2,1173,"Air Canada, Etihad Airways"
2,2022-05-30,YVR,AMD,12:20 a.m.,4:25 a.m.,39h 35m,2,1173,"Air Canada, Etihad Airways"
3,2022-05-30,YVR,AMD,8:00 p.m.,7:55 p.m.,35h 25m,2,1464,"Air Canada, Vistara"
4,2022-05-30,YVR,AMD,1:00 p.m.,8:35 p.m.,43h 05m,2,1548,"Air Canada, Air India"
5,2022-05-30,YVR,AMD,2:00 p.m.,8:35 p.m.,42h 05m,2,1548,"Air Canada, Air India"
6,2022-05-30,YVR,AMD,3:00 p.m.,8:35 p.m.,41h 05m,2,1548,"Air Canada, Air India"
7,2022-05-30,YVR,AMD,5:35 p.m.,8:35 p.m.,38h 30m,2,1548,"Air Canada, Air India"
8,2022-05-30,YVR,AMD,10:25 p.m.,8:35 p.m.,33h 40m,2,1548,"Air Canada, Air India"
9,2022-05-30,YVR,AMD,12:20 a.m.,8:35 p.m.,31h 45m,2,1548,"Air Canada, Air India"
10,2022-05-30,YVR,AMD,10:40 a.m.,2:15 a.m.,27h 05m,2,1611,"Alaska Airlines, Qatar Airways"


In [26]:
source_airport="YYZ"
destination_airport="BLR"
doj='2022-05-30'
url="https://www.ca.kayak.com/flights/" +str(source_airport) +"-" +str(destination_airport)+ "/"+str(doj)+ "?sort=bestflight_a"
driver = webdriver.Chrome(ChromeDriverManager().install())
driver.maximize_window()
driver.get(url)
time.sleep(15)
def click_more():

    for val in range(0,50):
        try:
            driver.find_element_by_xpath("//a[@class='moreButton']").click()
            time.sleep(4)
            val = val +1

        except:
            pass
click_more()



====== WebDriver manager ======
Current google-chrome version is 99.0.4844
Get LATEST chromedriver version for 99.0.4844 google-chrome
Driver [C:\Users\Haet\.wdm\drivers\chromedriver\win32\99.0.4844.51\chromedriver.exe] found in cache
<ipython-input-26-dae93289739f>:5: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())
<ipython-input-26-dae93289739f>:13: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  driver.find_element_by_xpath("//a[@class='moreButton']").click()


In [27]:
departure_time=[]
departure_time= driver.find_elements_by_xpath("//ol[@class='flights']//span[@class='depart-time base-time']")

arrival_time=[]
arrival_time= driver.find_elements_by_xpath("//ol[@class='flights']//span[@class='arrival-time base-time']")

duration =[]
duration  = driver.find_elements_by_xpath("//ol[@class='flights']//div[@class='section duration allow-multi-modal-icons']//div[@class='top']")

no_stop = []
no_stop  = driver.find_elements_by_xpath("//ol[@class='flights']//div[@class='section stops']//div[@class='top']")

price =[]
price = driver.find_elements_by_xpath("//div[@class = 'booking']")

airlines = []
airlines = driver.find_elements_by_xpath("//span[@class = 'codeshares-airline-names']")

<ipython-input-27-b14ccfb4e37d>:2: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  departure_time= driver.find_elements_by_xpath("//ol[@class='flights']//span[@class='depart-time base-time']")
<ipython-input-27-b14ccfb4e37d>:5: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  arrival_time= driver.find_elements_by_xpath("//ol[@class='flights']//span[@class='arrival-time base-time']")
<ipython-input-27-b14ccfb4e37d>:8: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  duration  = driver.find_elements_by_xpath("//ol[@class='flights']//div[@class='section duration allow-multi-modal-icons']//div[@class='top']")
<ipython-input-27-b14ccfb4e37d>:11: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  no_stop  = driver.find_elements_by_xpath("//ol[@class='flights']//div[@class='section stops']

In [29]:
len(no_stop)

117

In [30]:
stops=[]

In [25]:
type(no_stop[1])

selenium.webdriver.remote.webelement.WebElement

In [31]:
for i in range(len(departure_time)):
    no_stop[i] = no_stop[i].get_attribute('innerHTML')
    l = re.findall("(direct|1|2|3|4|5)",no_stop[i])[0]
    stops.append(l)